In [ ]:
!pip install shexer

# shape maps

Shape maps are an element used in ShEx to indicate which nodes should validated with which shapes.

In sheXer, we use shape maps for indicating which nodes should be used to extrac which shapes.

Even shape maps does not exist in SHACL, here you can use them to generated SHACL outputs.

There are three types of node selectors:
* Single nodes
* Focus expressions
* SPARQL queries

Single nodes and focus expressions are defined in shape map's spec:
[https://shex.io/shape-map/](https://shex.io/shape-map/)

SPARQL node selectors are a extension currently not included in the spec but supported by sheXer


In [16]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE, FIXED_SHAPE_MAP, JSON, SHACL_TURTLE
import requests

def remote_to_local(url, local_path):
  response = requests.get(url)
  if response.status_code == 200:
      with open(local_path, "w", encoding="utf-8") as out_stream:
          out_stream.write(response.text)

def default_namespaces():
    return {"http://example.org/": "ex",
            "http://www.w3.org/XML/1998/namespace/": "xml",
            "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
            "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
            "http://www.w3.org/2001/XMLSchema#": "xsd",
            "http://xmlns.com/foaf/0.1/": "foaf"
            }

In [ ]:
# In this case, I'm extracting a single shape using as example a single node

remote_to_local("https://raw.githubusercontent.com/weso/shexer/refs/heads/master/test/t_files/t_graph_1.ttl",
                "local_file.ttl")


shape_map = "<http://example.org/Jimmy>@<Person>"  # Fixed shape map, as defined in shape map's spec

shaper = Shaper(graph_file_input="./local_file.ttl",
                namespaces_dict=default_namespaces(),
                all_classes_mode=False,
                input_format=TURTLE,
                shape_map_raw=shape_map,  # Parameter to provide a the shape map
                shape_map_format=FIXED_SHAPE_MAP  # Parameter to provide shape map's format. This is the default value
                )
str_result = shaper.shex_graph(string_output=True)
print(str_result)

print("--------------")

str_result = shaper.shex_graph(string_output=True, output_format=SHACL_TURTLE)  # Same computation, but results in SHACL
print(str_result)



In [ ]:
# This is the very same case, but we are using JSON syntax for the shape map instead

shape_map = '[{"nodeSelector" : "<http://example.org/Jimmy>", "shapeLabel": "<Person>"}]'
shaper = Shaper(graph_file_input="./local_file.ttl",
                namespaces_dict=default_namespaces(),
                all_classes_mode=False,
                input_format=TURTLE,
                shape_map_raw=shape_map,  # Still using the same parameter to provide the shape map
                shape_map_format=JSON  # All other settings remain the same except for this
                )
str_result = shaper.shex_graph(string_output=True)
print(str_result)

In [ ]:
# In case you are using URIs whose namespace is included in the namespaces list,
# you can write less verbose shape maps, both fixed and in JSON syntax.
# Prefixes will be understood

shape_map = '[{"nodeSelector" : "ex:Jimmy", "shapeLabel": "<Person>"}]'
shaper = Shaper(graph_file_input="./local_file.ttl",
                namespaces_dict=default_namespaces(),  # In this list, the "ex" prefix is defined
                all_classes_mode=False,
                input_format=TURTLE,
                shape_map_raw=shape_map,
                shape_map_format=JSON
                )
str_result = shaper.shex_graph(string_output=True)
print(str_result)

In [ ]:
# Alternative node selector. In this case, we are using FOCUS expressions.
# Here we are extracting a shape <Person> using all instances of foaf:Person

shape_map = '[{"nodeSelector" : "{FOCUS a foaf:Person}", "shapeLabel": "<Person>"}]'
shaper = Shaper(graph_file_input="./local_file.ttl",
                namespaces_dict=default_namespaces(),
                all_classes_mode=False,
                input_format=TURTLE,
                shape_map_raw=shape_map,
                shape_map_format=JSON
                )
str_result = shaper.shex_graph(string_output=True)
print(str_result)

str_result = shaper.shex_graph(string_output=True)
print(str_result)

In [ ]:
# In this case, a shapa called <WithName> for every node having a foaf:name

shape_map = '[{"nodeSelector" : "{FOCUS foaf:name _}", "shapeLabel": "<WithName>"}]'
shaper = Shaper(graph_file_input="./local_file.ttl",
                namespaces_dict=default_namespaces(),
                all_classes_mode=False,
                input_format=TURTLE,
                shape_map_raw=shape_map,
                shape_map_format=JSON
                )
str_result = shaper.shex_graph(string_output=True)
print(str_result)

In [ ]:
# Now we are using a feature of shape maps which is not currently part of the standard
# sheXer allows for defining node selectors which are SPARQL queries.
# It works both in ShEx and SHACL.

# Any query will work as long as its result is single-column

shape_map = '[{"nodeSelector" : "SPARQL \'select ?p where { ?p a foaf:Person }\'", "shapeLabel": "<Person>"}]'
shaper = Shaper(graph_file_input="./local_file.ttl",
                namespaces_dict=default_namespaces(),
                all_classes_mode=False,
                input_format=TURTLE,
                shape_map_raw=shape_map,
                shape_map_format=JSON
                )
str_result = shaper.shex_graph(string_output=True)
print(str_result)

print("------------------------------------")

str_result = shaper.shex_graph(string_output=True, output_format=SHACL_TURTLE)
# Note that, in SHACL outputs, the targets both for FOCUS expressions selectors
# and SPARQL selectors will be expressed with N sh:targetNode declarations.
# In both cases, those nodes are the results of the query or the FOCUS expression
print(str_result)